<a href="https://colab.research.google.com/github/ayushi-3536/inFairness/blob/main/examples/compas_recidivism_prediction/compas_recidivism_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install inFairness requests


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 21.4 MB 1.2 MB/s 
     |████████████████████████████████| 664 kB 60.4 MB/s 
     |████████████████████████████████| 750.6 MB 13 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1+cu113 requires torch==1.12.1, but you have torch 1.11.0 which is incompatible.
torchtext 0.13.1 requires torch==1.12.1, but you have torch 1.11.0 which is incompatible.
torchaudio 0.12.1+cu113 requires torch==1.12.1, but you have torch 1.11.0 which is incompatible.


In [31]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from inFairness.fairalgo import SenSeI
from inFairness import distances
from inFairness.auditor import SenSRAuditor, SenSeIAuditor

%load_ext autoreload
%autoreload 2

import metrics

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
!git clone https://github.com/ayushi-3536/inFairness.git

Cloning into 'inFairness'...
remote: Enumerating objects: 343, done.
remote: Counting objects: 100% (128/128), done.
remote: Compressing objects: 100% (91/91), done.
remote: Total 343 (delta 75), reused 40 (delta 36), pack-reused 215
Receiving objects: 100% (343/343), 1.46 MiB | 27.20 MiB/s, done.
Resolving deltas: 100% (132/132), done.


In [4]:
cd inFairness

/content/inFairness


In [5]:
!pip install -e .


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/inFairness
  Attempting uninstall: inFairness
    Found existing installation: inFairness 0.2.2
    Uninstalling inFairness-0.2.2:
      Successfully uninstalled inFairness-0.2.2
  Running setup.py develop for inFairness


In [6]:
cd examples/compas_recidivism_prediction/

/content/inFairness/examples/compas_recidivism_prediction


In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from inFairness.fairalgo import SenSeI
from inFairness import distances
from inFairness.auditor import SenSRAuditor, SenSeIAuditor

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
class Dataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __getitem__(self, idx):
        data = self.data[idx]
        label = self.labels[idx]
        return data, label
    
    def __len__(self):
        return len(self.labels)



In [14]:
def get_input_output_df(data,target):

    cols = sorted(data.columns)
    output_col = target
    input_cols = [col for col in cols if col not in output_col]

    df_X = data[input_cols]
    df_Y = data[output_col]
    
    return df_X, df_Y

In [18]:
import os
import requests
import zipfile
import pandas as pd
from sklearn.preprocessing import StandardScaler

compas= 'https://github.com/propublica/compas-analysis/raw/master/compas-scores-two-years.csv'
data_df = pd.read_csv(compas,index_col=0,parse_dates=[0])
cols_to_drop = [
        'name','first','last','compas_screening_date','dob','age_cat',
        'v_screening_date','in_custody','out_custody','start','end',
        'c_jail_in', 'c_jail_out', 'c_case_number', 'c_offense_date',
       'c_arrest_date', 'c_days_from_compas',
       'c_charge_desc', 'is_recid', 'r_case_number', 
       'r_days_from_arrest', 'r_offense_date', 'r_charge_desc', 'r_jail_in',
       'r_jail_out', 'violent_recid', 'is_violent_recid', 'vr_case_number',
       'vr_charge_degree', 'vr_offense_date', 'vr_charge_desc',
       'screening_date','days_b_screening_arrest',
       'v_type_of_assessment', 'v_decile_score', 'v_score_text','event'
    ]

data_df.drop(cols_to_drop, axis=1, inplace=True)


categorical_vars = [
        'sex', 'race', 'c_charge_degree','r_charge_degree', 
        'type_of_assessment', 'score_text'
    ]
data_df = pd.get_dummies(data_df, columns=categorical_vars)
cols_to_drop=['race_Asian', 'race_Caucasian', 'race_Hispanic', 'race_Native American',
       'race_Other','sex_Female']

data_df.drop(cols_to_drop, axis=1, inplace=True)
train_data = data_df.sample(frac=0.8, random_state=123)
test_data = data_df.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)
continuous_vars = [
    'juv_fel_count', 'decile_score', 'juv_misd_count',
    'juv_other_count', 'priors_count','decile_score.1', 
    'priors_count.1'
    ]
scaler = StandardScaler().fit(train_data[continuous_vars])
train_data[continuous_vars] = scaler.transform(train_data[continuous_vars])
test_data[continuous_vars] = scaler.transform(test_data[continuous_vars])

train_data = get_input_output_df(train_data,'two_year_recid')
test_data = get_input_output_df(test_data,'two_year_recid')

X_train_df, Y_train_df = train_data
X_test_df, Y_test_df = test_data




In [22]:
def convert_df_to_tensor(data_X_df, data_Y_df):

    data_X = torch.tensor(data_X_df.values).float()
    data_Y = torch.tensor(data_Y_df.values)

    return data_X, data_Y

In [23]:
protected_vars = ['sex_Male', 'race_African-American']
protected_idxs = [X_train_df.columns.get_loc(var) for var in protected_vars]
X_train, y_train = convert_df_to_tensor(X_train_df, Y_train_df)
X_test, y_test = convert_df_to_tensor(X_test_df, Y_test_df)
X_train_df, Y_train_df = train_data
X_test_df, Y_test_df = test_data
train_ds = Dataset(X_train, y_train)
test_ds = Dataset(X_test, y_test)
# Create train and test dataloaders
train_dl = DataLoader(train_ds, batch_size=64, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=1000, shuffle=False)


In [24]:
# Create a fully connected neural network

class Model(nn.Module):

    def __init__(self, input_size, output_size):

        super().__init__()
        self.fc1 = nn.Linear(input_size, 100)
        self.fc2 = nn.Linear(100, 100)
        self.fcout = nn.Linear(100, output_size)

    def forward(self, x):

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fcout(x)
        return x

In [28]:
import metrics
device = torch.device('cpu')
input_size = X_train.shape[1]
output_size = 2
network_standard = Model(input_size, output_size).to(device)
optimizer = torch.optim.Adam(network_standard.parameters(), lr=1e-3)
loss_fn = F.cross_entropy

EPOCHS = 10
network_standard.train()
for epoch in range(EPOCHS):

      for x, y in train_dl:

          x, y = x.to(device), y.to(device)
          optimizer.zero_grad()
          y_pred = network_standard(x).squeeze()
          loss = loss_fn(y_pred, y)
          loss.backward()
          optimizer.step()

      if epoch % 10 == 0:
          print(f'{epoch} completed')
accuracy = metrics.accuracy(network_standard, test_dl, device)
print(f'Accuracy: {accuracy}')

0 completed
Accuracy: 0.9708939790725708


In [29]:
### Individually fair training
network_fair = Model(input_size, output_size).to(device)
optimizer = torch.optim.Adam(network_fair.parameters(), lr=1e-3)
lossfn = F.cross_entropy

distance_x = distances.SVDSensitiveSubspaceDistance()
distance_y = distances.SquaredEuclideanDistance()

distance_x.fit(X_train, n_components=10)
distance_y.fit(num_dims=output_size)

distance_x.to(device)
distance_y.to(device)

rho = 5.0
eps = 0.1
auditor_nsteps = 100
auditor_lr = 1e-3

fairalgo = SenSeI(network_fair, distance_x, distance_y, lossfn, rho, eps, auditor_nsteps, auditor_lr)

fairalgo.train()

for epoch in range(EPOCHS):
    for x, y in train_dl:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        result = fairalgo(x, y)
        result.loss.backward()
        optimizer.step()
    
    if epoch % 10 == 0:
        print(f'{epoch} completed')

accuracy = metrics.accuracy(network_fair, test_dl, device)

print(f'Accuracy: {accuracy}')

0 completed
Accuracy: 0.8967428803443909


In [ ]:
# Auditing using the SenSR Auditor

audit_nsteps = 1000
audit_lr = 0.1

auditor = SenSRAuditor(loss_fn=loss_fn, distance_x=distance_x, num_steps=audit_nsteps, lr=audit_lr, max_noise=0.5, min_noise=-0.5)

audit_result_stdmodel = auditor.audit(network_standard, X_test, y_test, lambda_param=10.0, audit_threshold=1.15)
audit_result_fairmodel = auditor.audit(network_fair, X_test, y_test, lambda_param=10.0, audit_threshold=1.15)

print("="*100)
print(f"Loss ratio (Standard model) : {audit_result_stdmodel.lower_bound}. Is model fair: {audit_result_stdmodel.is_model_fair}")
print(f"Loss ratio (fair model) : {audit_result_fairmodel.lower_bound}. Is model fair: {audit_result_fairmodel.is_model_fair}")
print("-"*100)
print("\t As signified by these numbers, the fair model is fairer than the standard model")
print("="*100)

/usr/local/lib/python3.7/dist-packages/inFairness/auditor/auditor.py:54: RuntimeWarning: invalid value encountered in true_divide


Loss ratio (Standard model) : 542.5004962436053. Is model fair: False
Loss ratio (fair model) : 1.0210911146684414. Is model fair: True
----------------------------------------------------------------------------------------------------
	 As signified by these numbers, the fair model is fairer than the standard model


In [30]:
# Auditing using the SenSeI Auditor

audit_nsteps = 500
audit_lr = 0.001

auditor = SenSeIAuditor(distance_x=distance_x, distance_y=distance_y, num_steps=audit_nsteps, lr=audit_lr, max_noise=0.5, min_noise=-0.5)

audit_result_stdmodel = auditor.audit(network_standard, X_test, y_test, loss_fn, audit_threshold=1.15, lambda_param=50.0)
audit_result_fairmodel = auditor.audit(network_fair, X_test, y_test, loss_fn, audit_threshold=1.15, lambda_param=50.0)

print("="*100)
print(f"Loss ratio (Standard model) : {audit_result_stdmodel.lower_bound}. Is model fair: {audit_result_stdmodel.is_model_fair}")
print(f"Loss ratio (fair model) : {audit_result_fairmodel.lower_bound}. Is model fair: {audit_result_fairmodel.is_model_fair}")
print("-"*100)
print("\t As signified by these numbers, the fair model is fairer than the standard model")
print("="*100)

/usr/local/lib/python3.7/dist-packages/inFairness/auditor/auditor.py:54: RuntimeWarning: invalid value encountered in true_divide


Loss ratio (Standard model) : 358036.5740139965. Is model fair: False
Loss ratio (fair model) : 1.0036988178130457. Is model fair: True
----------------------------------------------------------------------------------------------------
	 As signified by these numbers, the fair model is fairer than the standard model
